<a href="https://colab.research.google.com/github/rcofrer/Coursera_Capstone/blob/master/Off_shore_office_placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction 

As the COVID-19 spreads, restrictions rise and the world went from commuting everyday to work to a forced home office work style. The economy of several countries suddenly stopped and this hurt small and big companies. However, one advantage big companies have (multinationals) is that they can quickly move their work force to countries where is cheaper for them to hire workers. As an effort to make the off shores offers attractive for the locals, big corporations are trying to locate their new offices in strategic parts of town that will offer their workers proximity to venues that they conveniently could use in their way to or from the office. One popular destination for big corporation off shore activities in Europe is Poland. This analysis will be focus in the region of Lesser Poland(Malopolska), where there are many off shore offices from big companies like HBSC or UBS. 

# Data

## Geographical Data

For the purposes stated in the previous section the corresponding data set for the analysis will be information about different types of administrative division of the region Lesser Poland. 

In [0]:
import requests

url = 'https://data.opendatasoft.com/api/records/1.0/search/?dataset=geonames-postal-code%40public-us&q=Poland&rows=1093&sort=place_name&facet=country_code&facet=admin_name1&facet=admin_name2&facet=admin_name3&refine.country_code=PL&refine.admin_name1=Lesser+Poland'
results = requests.get(url)

In [0]:
import pandas as pd

# the key 'records' has the information needed for the region of Lesser Poland
results.json()['records']

df = pd.DataFrame(results.json()['records'])

In [26]:
df.columns

Index(['datasetid', 'recordid', 'fields', 'geometry', 'record_timestamp'], dtype='object')

In [27]:
# info that will be used 
df['fields'].iloc[3]

{'accuracy': '6',
 'admin_code1': '77',
 'admin_code2': '1203',
 'admin_code3': '120301',
 'admin_name1': 'Lesser Poland',
 'admin_name2': 'Powiat chrzanowski',
 'admin_name3': 'Alwernia',
 'coordinates': [50.0627, 19.5387],
 'country_code': 'PL',
 'latitude': '50.0627',
 'longitude': '19.5387',
 'place_name': 'Alwernia',
 'postal_code': '32-566'}

In [28]:
# Creating 
malopolska_data = pd.DataFrame(columns=['County','District','PlaceName','PostalCode','Latitude','Longitude'])
malopolska_data

,County,District,PlaceName,PostalCode,Latitude,Longitude


In [0]:
malopolska_data['County'] = df.apply(lambda x: x['fields']['admin_name2'],axis=1)
malopolska_data['District'] = df.apply(lambda x: x['fields']['admin_name3'],axis=1)
malopolska_data['PlaceName'] = df.apply(lambda x: x['fields']['place_name'],axis=1)
malopolska_data['PostalCode'] = df.apply(lambda x: x['fields']['postal_code'],axis=1)
malopolska_data['Latitude'] = df.apply(lambda x: x['fields']['latitude'],axis=1)
malopolska_data['Longitude'] = df.apply(lambda x: x['fields']['longitude'],axis=1)

In [30]:
malopolska_data.dtypes
# Need to change types of Latitude and Longitude data

County        object
District      object
PlaceName     object
PostalCode    object
Latitude      object
Longitude     object
dtype: object

In [31]:
malopolska_data['Latitude'] = pd.to_numeric(malopolska_data['Latitude'])
malopolska_data['Longitude'] = pd.to_numeric(malopolska_data['Longitude'])
malopolska_data.dtypes

County         object
District       object
PlaceName      object
PostalCode     object
Latitude      float64
Longitude     float64
dtype: object

The following shows the DataFrame complete

In [32]:
malopolska_data

,County,District,PlaceName,PostalCode,Latitude,Longitude
0,Powiat dąbrowski,Olesno,Adamierz,33-210,50.2031,20.8995
1,Powiat miechowski,Gmina Gołcza,Adamowice,32-075,50.3797,19.8553
2,Powiat krakowski,Zabierzów,Aleksandrowice,32-084,50.0838,19.7608
3,Powiat chrzanowski,Alwernia,Alwernia,32-566,50.0627,19.5387
4,Powiat wadowicki,Andrychów,Andrychów,34-120,49.8529,19.3403
...,...,...,...,...,...,...
1088,Kraków,Kraków,Kraków,31-573,50.0577,19.9978
1089,Kraków,Kraków,Kraków,31-579,50.0724,20.0086
1090,Kraków,Kraków,Kraków,31-640,50.0970,20.0211
1091,Kraków,Kraków,Kraków,31-702,50.0969,20.0582


In [33]:
malopolska_data.shape

(1093, 6)

In [34]:
malopolska_data[malopolska_data['PostalCode']=='38-340']

,County,District,PlaceName,PostalCode,Latitude,Longitude
73,Powiat gorlicki,Biecz,Biecz,38-340,49.7349,21.2595
95,Powiat gorlicki,Biecz,Binarowa,38-340,49.7558,21.2208
496,Powiat gorlicki,Biecz,Głęboka,38-340,49.7192,21.3154
604,Powiat gorlicki,Biecz,Grudna Kępska,38-340,49.7367,21.3045
892,Powiat gorlicki,Biecz,Korczyna,38-340,49.7124,21.2514


It can be seen in the previous tables that there are places that have the same County, District and place name, but they differ in postal code; and places that have the same County, District and postal code, but they differ in the place name. these situation point to different neighborhoods in the same places. For these reasons a label for a neighborhood will be compose of the place name and its postal code, to be able to fully differentiate each neighborhood.

In [0]:
malopolska_data['neighborhood']= malopolska_data['PlaceName']+','+ malopolska_data['PostalCode']
malopolska_data.drop(['PlaceName','PostalCode'],axis=1,inplace=True)

In [36]:
malopolska_data = malopolska_data[['County','District','neighborhood','Latitude','Longitude']]
malopolska_data

,County,District,neighborhood,Latitude,Longitude
0,Powiat dąbrowski,Olesno,"Adamierz,33-210",50.2031,20.8995
1,Powiat miechowski,Gmina Gołcza,"Adamowice,32-075",50.3797,19.8553
2,Powiat krakowski,Zabierzów,"Aleksandrowice,32-084",50.0838,19.7608
3,Powiat chrzanowski,Alwernia,"Alwernia,32-566",50.0627,19.5387
4,Powiat wadowicki,Andrychów,"Andrychów,34-120",49.8529,19.3403
...,...,...,...,...,...
1088,Kraków,Kraków,"Kraków,31-573",50.0577,19.9978
1089,Kraków,Kraków,"Kraków,31-579",50.0724,20.0086
1090,Kraków,Kraków,"Kraków,31-640",50.0970,20.0211
1091,Kraków,Kraków,"Kraków,31-702",50.0969,20.0582


Let us visualize the data extracted with the API.

In [0]:
import folium
# create map of New York using latitude and longitude values
# Coordinates of Malopolska region
latitude    =49.7225
longitude   =20.2503
map_malopolska = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, neighborhood  in zip(malopolska_data['Latitude'], malopolska_data['Longitude'], malopolska_data['neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_malopolska)  

In [38]:
map_malopolska

Now with this geographical information it is possible to extract venues information near to these locations.

## Venues Data 

In [0]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200505' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [0]:
def getNearbyVenues(names, pcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, pcode, lat, lng in zip(names, pcodes, latitudes, longitudes):
        print(name+pcode)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name+pcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
venues_list=[]
names       = malopolska_data['neighborhood']
latitudes   = malopolska_data['Latitude']
longitudes  = malopolska_data['Longitude']
radius      = 500

for name, lat, lng in zip(names, latitudes, longitudes):
    print(name)
            
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
    # return only relevant information for each nearby venue
    venues_list.append([(
        name, 
        lat, 
        lng, 
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

Adamierz,33-210
Adamowice,32-075
Aleksandrowice,32-084
Alwernia,32-566
Andrychów,34-120
Andrzejówka,33-370
Antolka,32-210
Augustyny,34-607
Augustyny,34-608
Babica,34-103
Babice,32-551
Babice,32-600
Bachowice,34-115
Bachowice,34-116
Bącza-Kunina,33-335
Baczków,32-708
Baczyn,34-211
Baczyn,32-084
Bałażówka,34-600
Balice,32-083
Balice,30-083
Balice,32-080
Balin,32-500
Banica,38-315
Bańska Niżna,34-424
Bańska Wyżna,34-424
Baranówka,32-010
Barbarka,32-046
Barcice,33-342
Barcice Dolne,33-342
Barcice Dolne,33-340
Barczków,32-813
Barnowiec,33-336
Bartkowa Posadowa,33-318
Bartne,38-307
Barwałd Dolny,34-124
Barwałd Górny,34-130
Barwałd Średni,34-124
Barwałd Średni,34-142
Batowice,32-087
Batowice,32-086
Bębło,32-089
Bębło,32-085
Bęczarka,32-444
Bęczyn,34-113
Będkowice,32-085
Będkowice,32-089
Bednarka,38-305
Będzieszyna,32-860
Bełdno,32-731
Berest,33-380
Bessów,32-711
Biadoliny Radłowskie,32-828
Biadoliny Radłowskie,32-830
Biadoliny Szlacheckie,32-828
Biadoliny Szlacheckie,32-854
Biała,33-150
Biała

In [45]:
nearby_venues['Neighborhood'].nunique()

336

In [0]:
malopolska_venues = nearby_venues
# malopolska_venues.to_csv('/content/malopolska_venues.csv')

In [0]:
# malopolska_venus = pd.read_csv('malopolska_venues.csv')

Let us check some entries of the DataFrame with the nearby venues.

In [52]:
print(malopolska_venues.shape)
malopolska_venues.head()

(2016, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Aleksandrowice,32-084",50.0838,19.7608,Dolina Aleksandrowicka,50.079756,19.760396,Scenic Lookout
1,"Alwernia,32-566",50.0627,19.5387,Marek Siewiorek Alverna It-group,50.062971,19.541813,Hotel
2,"Alwernia,32-566",50.0627,19.5387,Piekarnia - Sklep Alwernia Zaborowscy,50.062799,19.542400,Bakery
3,"Alwernia,32-566",50.0627,19.5387,Biedronka,50.059437,19.539711,Grocery Store
4,"Andrychów,34-120",49.8529,19.3403,Restauracja Adria,49.854911,19.342249,Eastern European Restaurant


Let us see how many venues are returned per neighborhood

In [54]:
malopolska_venues.groupby(by='Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Aleksandrowice,32-084",1,1,1,1,1,1
"Alwernia,32-566",3,3,3,3,3,3
"Andrychów,34-120",5,5,5,5,5,5
"Andrzejówka,33-370",1,1,1,1,1,1
"Balice,30-083",1,1,1,1,1,1
...,...,...,...,...,...,...
"Kraków,33-014",30,30,30,30,30,30
"Kępa Bogumiłowicka,33-100",1,1,1,1,1,1
"Kęty,31-650",2,2,2,2,2,2


And let us find out how many unique categories exists in the region

In [55]:
print('There are {} uniques categories.'.format(len(malopolska_venues['Venue Category'].unique())))

There are 211 uniques categories.
